# Saxena Carbon Database 
Required system packages and initialization

In [ ]:
import pandas as pd
import numpy as np
import sympy as sym
import os
import thermoengine

import importlib
sym.init_printing()


import matplotlib.pyplot  as plt
from fuzzywuzzy import fuzz
# python-Levenshtein

# Create coder module for pure phase using Berman Std State

In [ ]:
%run core.ipynb

In [ ]:
%run Saxena-std-state-model.ipynb

In [ ]:
def Saxena_params(Phase='Name', Formula='', 
                  H_TrPr=-1e6, S_TrPr=1e2, V_TrPr=3,
                  a_cp=100, b_cp=-1.0e-2, c_cp=3.0e6, d_cp=1.0e-6,
                  e_cp=0.0, g_cp=-1.0e3, h_cp=0.0,
                  alpha=1.0E-5, K=100.0, K_P=4.0):
    
    param_dict = {
        'Phase': Phase, 'Formula': Formula,
        'T_r': 298.15, 'P_r': 1.0,
        'H_TrPr': H_TrPr, 'S_TrPr': S_TrPr, 'V_TrPr': V_TrPr,
        'a_cp': a_cp, 'b_cp': b_cp, 'c_cp': c_cp, 'd_cp': d_cp, 
        'e_cp':e_cp, 'g_cp':g_cp, 'h_cp':h_cp,
        'alpha':alpha, 'K':K, 'K_P':K_P}
    return param_dict

In [ ]:
phase_params = {}

In [ ]:
lookup_phase_info('diamond')

In [ ]:
phase_params['Dia'] = Saxena_params(
    Phase='Diamond', Formula='C(1)',
    H_TrPr=1900.0, S_TrPr=2.38, V_TrPr=0.3417,
    a_cp=98.45, b_cp=-3.655e-2, c_cp=1.217e6, d_cp=1.098e-5,
    e_cp=0.0, g_cp=-1.659e3, h_cp=0.0,
    alpha=0.2243e-4, K=444.0*1e4, K_P=1.9)

In [ ]:
lookup_phase_info('graphite')

In [ ]:
phase_params['Gr'] = Saxena_params(
    Phase='Graphite', Formula='C(1)',
    H_TrPr=0.0, S_TrPr=5.74, V_TrPr=0.5298,
    a_cp=60.86, b_cp=-1.024e-2, c_cp=7.139e5, d_cp=1.669e-6,
    e_cp=0.0, g_cp=-9.922e2, h_cp=0.0,
    alpha=0.3460e-4, K=33.8*1e4, K_P=8.9)

In [ ]:
modelDB = make_custom_database('saxena', phase_params)

In [ ]:
display(modelDB.phase_info)
Dia = modelDB.get_phase('Dia')
Gr = modelDB.get_phase('Gr')

In [ ]:
Dia.volume(298.15,1.0)

In [ ]:
Gr.volume(298.15,1)

In [ ]:
# Dia.param_names

In [ ]:
# Dia.get_param_values(param_names=Dia.param_names)

In [ ]:
Dia.entropy(298.15,1.0)

In [ ]:

T= np.linspace(240, 4000.0,2001)
Cp = Dia.heat_capacity(T, 1.0)

plt.figure()
plt.plot(T, Cp, 'k-')
plt.xlim(100,4000)

In [ ]:
rxn = modelDB.get_rxn(['Gr','Dia'], [0,0], [-1, +1])

In [ ]:
# rxn.affinity(298.15, 1)
rxn.affinity(298.15, 1)

In [ ]:
from scipy import optimize

In [ ]:
P = 3e4
P = np.linspace(1, 60e4,101)
Tlims = [10, 2500]

Tbnd = np.tile(np.nan, P.shape)

for ind, iP in enumerate(P):
    fun = lambda T, P=iP: rxn.affinity(T, P)
    try:
        Tbnd[ind]=optimize.brentq(fun, Tlims[0], Tlims[1])
    except:
        continue

In [ ]:
plt.figure()
plt.plot(P/1e4, Tbnd, 'k-')

plt.xlabel('Press. [GPa]')
plt.ylabel('Temp. [K]')

plt.text(40, 500,'Diamond')
plt.text(20, 2000,'Graphite')

In [ ]:
rxn.affinity(500, 30e4)

In [ ]:
rxn.affinity(300, .5e9)

In [ ]:

rxn.affinity(35, 1)
rxn.affinity(40, 1e3)
rxn.affinity(45, 1e4)
rxn.affinity(90, 5e4)
rxn.affinity(90, 1e5)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
P = 1.0
P = 1e3
# P = 3e3
# P = 1e4
# P = 3e4
P = 10.0e4
# P = 30.0e4
# P = 50.0e4
T = np.linspace(0.1, 4000, 2001)
plt.figure()
plt.plot(T, rxn.affinity(T, P), 'k-')
plt.plot(T, 0*T, 'k--')
plt.ylim(-1e4, +1e4)
P

In [ ]:

rxn.trace_boundary(Plims=[1,1e4], init_guess=35.0)

In [ ]:
# rxn.trace_boundary(Plims=[1,1e6], init_guess=3000.0)
# rxn.trace_boundary(Tlims=[300,1000])

In [ ]:
thermoengine.phases.FixedRxnSet?

In [ ]:
Dia = modelDB.get_phase('Dia')

In [ ]:
Dia.chem_potential(300.0, 30e3)

In [ ]:
import dill

In [ ]:
Cal = modelDB.get_phase('Cal')

In [ ]:
output = {}
output['Cal'] = Cal

In [ ]:
with open('test.pkl','wb') as file:
    dill.dump(output, file)